In [1]:
import os
import sys
if sys.path[-1] != "../":
    sys.path.append("../")
    os.chdir("../")

import re
import numpy as np
import pandas as pd
# import seaborn as sns
from IPython.display import display
from random import sample
from transformers import AutoModel, AutoTokenizer, T5ForConditionalGeneration

import torch
import torch.nn.functional as F
from models.AutoModel import AutoModel as AM
from utils.util import *
from utils.index import *
from utils.data import *

from hydra import initialize, compose

config = Config()
with initialize(version_base=None, config_path="../data/config/"):
    overrides = [
        # "base=NQ320k",
        "base=MSMARCO-doc",
        "++text_col=[2,1,3]",
        # "base=MS300k",
        # "++plm=t5",
    ]
    hydra_config = compose(config_name="_example", overrides=overrides)
    config._from_hydra(hydra_config)

loaders = prepare_data(config)

loader_text = loaders["text"]
loader_query = loaders["query"]
text_dataset = loader_text.dataset
query_dataset = loader_query.dataset

# train_dataset = prepare_train_data(config, loader_text.dataset)
# train_query_dataset = train_dataset.query_datasets[0]

[2024-01-24 06:26:26,143] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


[2024-01-24 06:26:28,876] INFO (Config) setting seed to 42...
[2024-01-24 06:26:28,882] INFO (Config) setting PLM to t5...
[2024-01-24 06:26:29,085] INFO (Config) Config: {'adam_beta1': 0.9, 'adam_beta2': 0.999, 'adam_epsilon': 1e-08, 'batch_size': 2, 'bf16': False, 'cache_root': 'data/cache/MSMARCO-doc', 'data_format': 'memmap', 'data_root': '/share/peitian/Data/Datasets/Adon', 'dataset': 'MSMARCO-doc', 'debug': False, 'deepspeed': None, 'device': 0, 'distill_src': 'none', 'early_stop_patience': 5, 'enable_all_gather': True, 'enable_distill': False, 'enable_inbatch_negative': True, 'epoch': 20, 'eval_batch_size': 2, 'eval_delay': 0, 'eval_flops': False, 'eval_metric': ['mrr', 'recall'], 'eval_metric_cutoff': [1, 5, 10, 100, 1000], 'eval_mode': 'retrieve', 'eval_posting_length': False, 'eval_set': 'dev', 'eval_step': '1e', 'fp16': False, 'grad_accum_step': 1, 'hits': 1000, 'index_shard': 32, 'index_thread': 10, 'index_type': 'invvec', 'learning_rate': 3e-06, 'load_ckpt': None, 'load_en

In [3]:
code_type = "term"
# code_type = "2gram"
# code_type = "id"
code_tokenizer = "t5"
# code_length = 26
code_length = 32

t = AutoTokenizer.from_pretrained(os.path.join(config.plm_root, code_tokenizer))

text_codes = np.memmap(
    f"data/cache/{config.dataset}/codes/{code_type}/{code_tokenizer}/{code_length}/codes.mmp",
    mode="r",
    dtype=np.int32
).reshape(len(text_dataset), -1).copy()

# trie = TrieIndex(save_dir=f"data/cache/{config.dataset}/codes/{code_type}/{code_tokenizer}/{code_length}")
# trie.load()

# wordset = WordSetIndex(save_dir=f"data/cache/{config.dataset}/codes/{code_type}/{code_tokenizer}/{code_length}", sep_token_id=6)
# wordset.fit(None)

# df = pd.DataFrame(text_codes)
# duplicates = df.groupby(df.columns.tolist(),as_index=False).size()
# duplicates = duplicates.sort_values("size", ascending=False)
# duplicates.reset_index(drop=True, inplace=True)

# dup = df.duplicated(keep=False).to_numpy()
# dup_indices = np.argwhere(dup)[:, 0]
# len(dup_indices)

In [14]:
# indices = random.sample(range(len(text_dataset)), 5)
# indices = range(10)
indices = [3088934]
# indices = dup_indices
text_code = text_codes[indices]
text_code[text_code == -1] = 0
# display(text_code)
display(t.batch_decode(text_code))
display(t.batch_decode(np.array(text_dataset[indices]["text"]["input_ids"])[:, :1500]))

# most_dup_idx = np.argwhere((text_codes == duplicates.iloc[0].to_numpy()[:-1]).all(-1))[:, 0]
# most_dup_code = text_codes[most_dup_idx]
# most_dup_code[most_dup_code == -1] = 0
# most_dup_text = np.array(text_dataset[most_dup_idx]["text"]["input_ids"])[:, :code_length + 5]
# t.batch_decode(most_dup_code), t.batch_decode(most_dup_text)

['<pad> kidney, foods, eat, creatinine, creatine, creatinine2013, blood, high, level, snack, mean,</s><pad>']

['http://www.kidney-treatment.org/creatinine/162.html What Foods Not to Eat When Having High Creatinine Font Size A A AWhat Foods Not to Eat When Having High Creatinine2013-10-04 10:31What foods not to eat when having high creatinine? To know well about the answer, we need to firstly understand what is creatinine and what high creatinine level mean. What is creatinine? Creatinine is a breakdown product of creatine, which is an important part of muscle. In normal condition, creatinine is filtered by glomerulus and then discharged with urine. Therefore, creatinine level in blood is always kept in a stable range 0.5-1.3mg/d L. What does high creatinine level mean? Since kidneys are responsible for discharging creatinine in blood, so when kidneys are injured for some reason, creatinine level in blood increases. Serum creatinine level reflects how well kidney functions, so patients are suggested to test their creatinine level in blood, so as to judge whether kidney function is affected. Cre

In [26]:
# get document container specific words
wordset = WordSetIndex(save_dir=f"data/cache/{config.dataset}/codes/{code_type}/{code_tokenizer}/{code_length}", sep_token_id=6)
wordset.fit(None)
word = t.encode("sodium")[:-1]
wordset.vocab[word + [6]]

[19049, 1]

In [17]:
# convert to LLM-Indexer

with open("/share/peitian/Data/Datasets/model-based-ir/msmarco-doc/docid.unicoil-term-comma-32.json", "w") as f:
    for i, code in enumerate(tqdm(text_codes)):
        code = code[1:]
        code = code[code != -1]
        code = t.decode(code, skip_special_tokens=True).strip(",")
        if len(code) == 0:
            print(i)
            break
            code = "Empty document"
        f.write(json.dumps({"docid": code}, ensure_ascii=False) + "\n")

100%|██████████| 3213835/3213835 [01:19<00:00, 40388.25it/s]


In [12]:
# check text_codes and text
positives = load_pickle(f"data/cache/{config.dataset}/dataset/query/dev/positives.pkl")
query_dataset = QueryDataset(config, "dev")
for k, v in positives.items():
    pos_index = v[0]
    query = query_dataset[k]["query"]["input_ids"]
    text = text_dataset[pos_index]["text"]["input_ids"]
    text_code = text_codes[pos_index]
    
    query = t.decode(query, skip_special_tokens=True)
    text = t.decode(text[:100], skip_special_tokens=True)
    text_code = t.decode(text_code[text_code != -1], skip_special_tokens=True)

    print("*"*50)
    print(f"{'Query': <50}: {query}")
    print(f"{'Text': <50}: {text}")
    print(f"{'Code': <50}: {text_code}")

    x = input()
    if x == "s":
        break

[2024-01-20 04:02:18,510] INFO (Dataset) initializing MSMARCO-doc memmap Query dev dataset...


**************************************************
Query                                             : does xpress bet charge to deposit money in your account
Text                                              : http://www.xpressbet.com/paynearme. We make wagering easy for you Online Log In to our safe and secure wagering environment for a fast and efficient wagering experience. Mobile Mobile wagering is available on most smartphones and tablets, making it easy to wager anytime, anywhere. Phone Just pick up the phone to speak with one of our Live Tellers or use our Automated Voice & Touch Tone services. Curious? See our interface
Code                                              : paynearme, wager, xpressbet, tellers, mobile, anytime, smartphones, easy, anywhere, tablets, secure,
**************************************************
Query                                             : how much is a cost to run disneyland
Text                                              : https://www.quora.

In [33]:
text_indices = [127020,  140340,  367753,  405272,  706774,  865285,  962994,
       1035936, 1095597, 1118707, 1250718, 1379532, 1794431, 1928216,
       1993257, 2049300, 2098233, 2199433, 2199760, 2204684, 2234871,
       2303640, 2327633, 2369164, 2459799, 2768104, 3112041]
texts = text_dataset[text_indices]["text"]['input_ids'][:, :100]
codes = text_codes[text_indices]
codes[codes == -1] = 0

t.batch_decode(texts, skip_special_tokens=True), t.batch_decode(codes, skip_special_tokens=True)

(['http://chemdemos.uoregon.edu/demos/Precipitation-of-Silver-Chloride Please read the following disclaimer carefully "Precipitation of Silver Chloride When clear, colorless silver nitrate solution is added to clear, colorless sodium chloride solution, white silver chloride precipitates. Curriculum Notes This demo is typically performed when solubility rules are being presented. Allow two minutes for',
  'http://www.myfitnesspal.com/food/calories/homemade-tomato-sandwich-2-slices-white-bread-1-fresh-tomato-12945968 Calories in Homemade Tomato Sandwich 2 Slices White Bread, 1 Fresh Tomato, Calories in Homemade Tomato Sandwich 2 Slices White Bread, 1 Fresh Tomato,Nutrition Facts Homemade Tomato Sandwich - 2 Slice',
  'https://www.quora.com/Why-is-a-yellow-color-more-common-in-city-street-lighting-than-white Why is a yellow color more common in city street lighting than white? Brandon Row, House design lover Photograph lover Answered Aug 20, 2017Most street lighting comes in the form of h

In [17]:
# check query_codes
model = "iter0"

query_sets = ["train"]
dyn_text_codes = []
qrels = []
query_datasets = []
for query_set in query_sets:
    qrel = load_pickle(f"data/cache/{config.dataset}/dataset/query/{query_set}/qrels.pkl")
    dyn_text_codes.append(
        np.memmap(
            f"data/cache/{config.dataset}/codes/{code_type}/{code_tokenizer}/{code_length}/{model}/{query_set}/codes.mmp",
            mode="r",
            dtype=np.int32
        ).reshape(len(qrel), -1, code_length).copy()
    )
    qrels.append(qrel)
    query_datasets.append(QueryDataset(config, query_set))

# text_idx: [[(qrel_idx, query_idx)], ...,]
tidx_2_qrel_query_idx_pair = defaultdict(lambda: [[] for _ in range(len(query_sets))])
for query_set_idx, qrel in enumerate(qrels):
    for j, rel in enumerate(qrel):
        tidx_2_qrel_query_idx_pair[rel[1]][query_set_idx].append((j, rel[0]))
# new_docs = {}
# for k,v in tidx_2_qrel_query_idx_pair.items():
#     has_common = 0
#     for x in v:
#         has_common += len(x) > 0
#     if has_common == len(query_sets):
#         new_docs[k] = v
# tidx_2_qrel_query_idx_pair = new_docs
same_count = 0
same = False
demo = 1

for tidx, qindices in tidx_2_qrel_query_idx_pair.items():
    text_code = text_codes[tidx]
    text_code[text_code == -1] = 0
    # print(f'******Text Code   ({tidx}):******\n{t.decode(text_code)}')
    if tidx != 3088934:
        continue

    for query_set_idx, query_set in enumerate(query_sets):
        query_dataset = query_datasets[query_set_idx]
        qindice = qindices[query_set_idx]

        for qrel_idx, qidx in qindice:
            dyn_text_code = dyn_text_codes[query_set_idx][qrel_idx]
            dyn_text_code[dyn_text_code == -1] = 0

            for c in dyn_text_code:
                if (c == text_code).all():
                    same_count += 1
                    same = True
                    # if demo:
                    #     print(f'******{query_set} Query ({qidx}):******\n{t.decode(query_dataset[qidx]["query"]["input_ids"])}')
                    #     print(f'******Text Code   ({tidx}):******\n{t.decode(text_code)}')
                    #     print(f"******Sorted Text Code for {query_set}:******\n{t.decode(dyn_text_code)}")
                else:
                    same = False
            if not same and demo:
                print(f'\n******{query_set} Query ({qidx}):******\n{t.decode(query_dataset[qidx]["query"]["input_ids"])}')
                print(f'******Text Code   ({tidx}):******\n{t.decode(text_code)}')
                print(f"******Sorted Text Code for {query_set}:******\n{t.batch_decode(dyn_text_code, skip_special_tokens=True)}")

                x = input()
                if x == "s":
                    raise StopIteration

same_count

[2024-01-24 07:33:19,308] INFO (Dataset) initializing MSMARCO-doc memmap Query train dataset...



******train Query (28337):******
what foods should not be eaten in kidney failure</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
******Text Code   (3088934):******
<pad> kidney, foods, eat, creatinine, creatine, creatinine2013, blood, high, level, snack, mean,</s><pad>
******Sorted Text Code for train:******
['kidney, foods, eat, creatinine, creatine, level, high, blood, creatinine2013, snack, mean,', 'kidney, eat, foods, creatinine, creatine, blood, creatinine2013, high, level, mean, snack,', 'kidney, foods, eat, blood, creatinine, creatine, level, creatinine2013, high, mean, snack,']


0

In [9]:
# cases that either res1 or res2 succeeds

model1_result = load_pickle(f"data/cache/MSMARCO-doc/retrieve/AutoTSG/dev/trie.pkl")
model2_result = load_pickle(f"data/cache/MSMARCO-doc/retrieve/AutoTSG/dev/set.pkl")

code_type1 = "term"
code_type2 = "term"
code_tokenizer = "t5"
code_length = 32

text_codes1 = np.memmap(
    f"data/cache/{config.dataset}/codes/{code_type1}/{code_tokenizer}/{code_length}/codes.mmp",
    mode="r",
    dtype=np.int32
).reshape(len(text_dataset), -1).copy()
text_codes2 = np.memmap(
    f"data/cache/{config.dataset}/codes/{code_type2}/{code_tokenizer}/{code_length}/codes.mmp",
    mode="r",
    dtype=np.int32
).reshape(len(text_dataset), -1).copy()

t = AutoTokenizer.from_pretrained(os.path.join(config.plm_root, code_tokenizer))

query_dataset = QueryDataset(config)

positives = load_pickle(f"data/cache/{config.dataset}/dataset/query/dev/positives.pkl")
positive_docs = {}

for k, v in positives.items():
    positive = v[0]
    positive_docs[positive] = k

query_sets = ["train"]
query_datasets = []
queries = defaultdict(list)
for query_set_idx, query_set in enumerate(query_sets):
    qrels = load_pickle(f"data/cache/{config.dataset}/dataset/query/{query_set}/qrels.pkl")
    for x in qrels:
        qidx = x[-2]
        tidx = x[-1]
        queries[tidx].append((query_set_idx, qidx))
    query_datasets.append(QueryDataset(config, query_set))

for qidx, positive in positives.items():
    # if qidx != 3315:
    #     continue
    positive = positive[0]
    model1_res = model1_result[qidx]
    model2_res = model2_result[qidx]
    query = query_dataset[qidx]["query"]["input_ids"]
    showcase = False
    if positive in model1_res and positive not in model2_res:
    # if (positive in model1_res and model1_res.index(positive) < 5) and (positive not in model2_res or model2_res.index(positive) > 5):
        text_code1 = text_codes1[positive]
        text_code2 = text_codes2[positive]
        false_pos_code = text_codes2[model2_res[0]]
        string = f"Model1 wins: {qidx, positive}"
        showcase = True

    elif positive in model2_res and positive not in model1_res:
        text_code1 = text_codes1[positive]
        text_code2 = text_codes2[positive]
        false_pos_code = text_codes1[model1_res[0]]
        string = f"Model2 wins: {qidx, positive}"
        showcase = True

    if showcase:
        has_train = False
        all_q = [[] for _ in query_sets]
        for query_set_idx, query_idx in queries[positive]:
            new_query = t.decode(query_datasets[query_set_idx][query_idx]["query"]["input_ids"], skip_special_tokens=True)
            all_q[query_set_idx].append(new_query)
        for j, new_query in enumerate(all_q):
            if len(new_query):
                print(f"{string:*^50}")
                print(f"{query_sets[j] + ' Queries': <50}: {new_query}")
                has_train = True
        if has_train:
            # print(f"{string:*^50}")
            print(f"{'Query': <50}: {t.decode(query, skip_special_tokens=True)}")
            print(f"{f'Target Anchor ({code_type1})': <50}: {t.decode(text_code1[text_code1 != -1], skip_special_tokens=True)}")
            # print(f"{f'Target Anchor ({code_type2})': <50}: {t.decode(text_code2[text_code2 != -1], skip_special_tokens=True)}")
            print(f"{'False Positive Anchor': <50}: {t.decode(false_pos_code[false_pos_code != -1], skip_special_tokens=True)}")
            # print(f"{f'Target Title ({code_type1})': <50}: {t.decode(text_dataset[positive]['text']['input_ids'][:20], skip_special_tokens=True)}")
            # print(f"{f'Negative Title  ({code_type2})': <50}: {t.batch_decode(text_dataset[model2_res]['text']['input_ids'][:, :20], skip_special_tokens=True)}")
            x = input()
            if x == "s":
                break

[2024-01-24 06:42:11,421] INFO (Dataset) initializing MSMARCO-doc memmap Query dev dataset...
[2024-01-24 06:42:12,315] INFO (Dataset) initializing MSMARCO-doc memmap Query train dataset...


************Model1 wins: (24, 2236571)************
train Queries                                     : ['what does it mean is a government job i']
Query                                             : government does do
Target Anchor (term)                              : government, what, politics, answer, job, laws, reference, charge, branches, forms, enforces, carries, romans,
False Positive Anchor                             : chile, government, chilean, type, president, answers, what, 44chile, govt, republic,
************Model2 wins: (34, 2108113)************
train Queries                                     : ['cost to send postcards', 'what is the price to mail a postcard']
Query                                             : how much is the stamp to send a card
Target Anchor (term)                              : postcard, card, postal, mail, cost, 49, send, post, much, 34, usps, stamp, inches, 6,
False Positive Anchor                             : postcard, card, postcards2, postca

In [ ]:
# cases for the failed dev queries and the corresponding train queries for a paticular model

dataset = "MSMARCO-passage"
retrieval_result = load_pickle(f"data/cache/{dataset}/retrieve/AutoTSG/dev/retrieval_result.pkl")
positives = load_pickle(f"data/cache/{dataset}/dataset/query/dev/positives.pkl")
query_dataset = QueryDataset(config)

miss_queries = {}
miss_docs = {}

success_queries = {}
success_docs = {}

for k, v in positives.items():
    positive = v[0]
    res = retrieval_result[k]
    if positive not in res:
        miss_queries[k] = positive
        miss_docs[positive] = k
    else:
        success_queries[k] = positive
        success_docs[positive] = k

query_sets = ["train"]
overlap_miss = defaultdict(list)
overlap_success = defaultdict(list)

query_datasets = []

for query_set_idx, query_set in enumerate(query_sets):
    qrels = load_pickle(f"data/cache/{config.dataset}/dataset/query/{query_set}/qrels.pkl")
    for x in qrels:
        qidx = x[-2]
        tidx = x[-1]
        if tidx in miss_docs:
            overlap_miss[tidx].append((query_set_idx, qidx))
        elif tidx in success_docs:
            overlap_success[tidx].append((query_set_idx, qidx))
    query_datasets.append(QueryDataset(config, query_set))

print(f"Mean query number for missed queries: {mean_len(overlap_miss.values())}\nMean query number for succeeded queries: {mean_len(overlap_success.values())}")

for k, v in miss_queries.items():
    # positive = v[0]
    positive = v

    query = query_dataset[k]["query"]["input_ids"]
    text = text_dataset[positive]["text"]["input_ids"]
    text_code = text_codes[positive]
    print(f"\n{f'Qidx: {k} Tidx: {positive}':*^40}")
    print(f"{'Query': <20}: {t.decode(query, skip_special_tokens=True)}")
    # print(f"{'Target':*^25}\n{t.decode(text, skip_special_tokens=True)}")
    print(f"{'Anchor': <20}: {t.decode(text_code[text_code != -1], skip_special_tokens=True)}")
    print(f"{'Text': <20}: {t.decode(text_dataset[positive]['text']['input_ids'][:100], skip_special_tokens=True)}")
    if positive in overlap_miss:
        qindices = overlap_miss[positive]
        queries = [[] for _ in query_sets]
        for query_set_idx, query_idx in qindices:
            queries[query_set_idx].append(t.decode(query_datasets[query_set_idx][query_idx]['query']['input_ids'], skip_special_tokens=True))
        for j, query in enumerate(queries):
            if len(query):
                print(f"{query_sets[j] + ' Queries': <20}: {query}")
    x = input()
    if x == "s":
        break

In [2]:
# create new query set based on an existing one

# dataset = "Top300k-filter"
# dataset = "NQ320k"
dataset = "MSMARCO-doc"
ori_query_set = "doct5"
query_set = "doct5-3"
k = 3

try:
    qid2idx = load_pickle(f"data/cache/{dataset}/dataset/query/{ori_query_set}/id2index.pkl")
except FileNotFoundError:
    qid2idx = {}
    with open(f"{config.data_root}/{dataset}/queries.{ori_query_set}.tsv") as f:
        for qidx, line in enumerate(tqdm(f, desc="Collecting qid2idx")):
            qid = line.split("\t")[0]
            qid2idx[qid] = qidx
            
tid2idx = load_pickle(f"data/cache/{dataset}/dataset/text/id2index.pkl")

qindices = []
tid2qrels = defaultdict(list)

train_positives = load_pickle(f"data/cache/{dataset}/dataset/query/train/positives.pkl")
train_positives = set([x[0] for x in train_positives.values()])
miss_docs = set(range(len(text_dataset))) - train_positives
print(f"number of documents missing in training set: {len(miss_docs)}")

with open(f"{config.data_root}/{dataset}/qrels.{ori_query_set}.tsv") as ori_qrel_file, open(f"{config.data_root}/{dataset}/qrels.{query_set}.tsv", "w") as qrel_file, open(f"{config.data_root}/{dataset}/queries.{ori_query_set}.tsv") as ori_query_file, open(f"{config.data_root}/{dataset}/queries.{query_set}.tsv", "w") as query_file:
    for i, line in enumerate(ori_qrel_file):
        qid, _, tid, _ = line.strip().split("\t")
        qidx = qid2idx[qid]

        # filter out the existing ones
        # tidx = tid2idx[tid]
        # if tidx in miss_docs:
        #     tid2qrels[tid].append(line)
        #     qindices.append(qidx)

        # keep the first k elements
        if len(tid2qrels[tid]) >= k:
            continue
        else:
            tid2qrels[tid].append(line)
            qindices.append(qidx)

    qindices = set(qindices)
    for i, line in enumerate(ori_query_file):
        if i in qindices:
            query_file.write(line)

    for qrels in tid2qrels.values():
        for line in qrels:
            qrel_file.write(line)

number of documents missing in training set: 2893908


In [ ]:
# false prune plot
model = "BOW_qg"
retrieval_result = load_pickle(f"data/cache/{config.dataset}/retrieve/{model}/dev/bm25-trie.pkl")
positives = load_pickle(f"data/cache/{config.dataset}/dataset/query/dev/positives.pkl")

# code_type = "words_comma_plus_stem"
# code_type = "title_comma-first"
code_type = "bm25_comma"
code_length = 34
code_tokenizer = "t5"
wordset = WordSetIndex(save_dir=f"data/cache/{config.dataset}/codes/{code_type}/{code_tokenizer}/{code_length}", sep_token_id=6)
wordset.fit(None)

all_missed = 0
for k, v in retrieval_result.items():
    positive = positives[k][0]
    if positive not in v:
        all_missed += 1
print(all_missed)

cutoffs = [1,2,3,4,5]
counts = [{"all_missed": 0, "false_prune": 0, "wordset_match": 0, "decode_step": i} for i in cutoffs]

docs = wordset.docs
stopwords = np.array([0, 1, -1])
skip_qindices = {}

for i, cutoff in enumerate(cutoffs):
    false_prune_count = 0
    wordset_match_count = 0

    for qidx, res in retrieval_result.items():
        if qidx in skip_qindices:
            continue

        positive = positives[qidx][0]
        if positive not in res:
            gt_doc = docs[positive]
            generated_doc = docs[res]
            # invalid_at_cutoff = ((gt_doc[cutoff] == -1) * (generated_doc[:, cutoff] == -1)).astype(bool)
            # print(invalid_at_cutoff)
            
            diff_at_cutoff = (gt_doc[cutoff] != generated_doc[:, cutoff])
            if diff_at_cutoff.all():
                false_prune_count += 1
                skip_qindices[qidx] = 1
                for gdoc in generated_doc:
                    overlap = np.intersect1d(gt_doc, gdoc[:cutoff])  # n
                    idx = (overlap[..., None] == stopwords[None, ...]).any(-1)
                    overlap = overlap[~idx]                 # <=n
                    if len(overlap):
                        wordset_match_count += 1
                        break

        
    if i == 0:
        counts[i]["all_missed"] = all_missed
    else:
        all_missed = all_missed - counts[i - 1]["false_prune"]
        counts[i]["all_missed"] = all_missed
    
    counts[i]["false_prune"] = false_prune_count
    counts[i]["wordset_match"] = wordset_match_count
    
print(counts)
data = pd.DataFrame(counts)
data.drop(columns=["all_missed"], inplace=True)
data = data.melt(id_vars=["decode_step"], value_vars=["false_prune", "wordset_match"], value_name="count")
ax = sns.barplot(data, x="decode_step", y="count", hue="variable")
ax.set_ylabel("#Error Query")
ax.set_xlabel("Decode Step")

In [ ]:
# check duplicated codes and corresponding train queries

dup_index = np.argwhere((text_codes == duplicates.loc[0].to_numpy()[:-1]).all(-1))[:, 0]

# get train queries
queries = [None for _ in range(len(dup_index))]
arange = np.arange(len(queries))
for x in train_dataset.qrels:
    if x[-1] in dup_index:
        idx = arange[dup_index == x[-1]][0]
        queries[idx] = t.decode(train_dataset.query_datasets[0][x[-2]]["query"]["input_ids"], skip_special_tokens=True)
print(len(dup_index))

j = 0
with open("/share/project/peitian/Data/Adon/Top300k/collection.tsv") as f:
    for i,line in enumerate(f):
        if i in dup_index:
            for k, v in positives.items():
                if idx in v:
                    print(f"{'Dev Query':*^20}\n{t.decode(query_dataset[k]['query']['input_ids'], skip_special_tokens=True)}")
            print(f"{'Train Query':*^20}\n{queries[j]}")
            print(line)
            j += 1
            x = input()
            if x == "s":
                break

In [ ]:
# cases that either res1 or res2 succeeds (different text_codes)

model1_result = load_pickle(f"data/cache/NQ/retrieve/BOW_doct5-miss-doc/dev/best.pkl")
model2_result = load_pickle(f"data/cache/NQ/retrieve/BOW_doct5-miss-doc/dev/50.pkl")

t = AutoTokenizer.from_pretrained(os.path.join(config.plm_root, code_tokenizer))

query_dataset = QueryDataset(config)

positives = load_pickle("data/cache/NQ/dataset/query/dev/positives.pkl")
positive_docs = {}

for k, v in positives.items():
    positive = v[0]
    positive_docs[positive] = k

query_sets = ["train-sub", "doct5-miss-sub"]
query_datasets = []
queries = defaultdict(list)
for query_set_idx, query_set in enumerate(query_sets):
    qrels = load_pickle(f"data/cache/{config.dataset}/dataset/query/{query_set}/qrels.pkl")
    for x in qrels:
        qidx = x[-2]
        tidx = x[-1]
        queries[tidx].append((query_set_idx, qidx))
    query_datasets.append(QueryDataset(config, query_set))

code_type1 = "words_comma"
code_length1 = 34
text_codes1 = np.memmap(
    f"data/cache/{config.dataset}/codes/{code_type1}/{code_tokenizer}/{code_length1}/codes.mmp",
    mode="r",
    dtype=np.int32
).reshape(len(text_dataset), -1).copy()

code_type2 = "words_comma"
code_length2 = 50
text_codes2 = np.memmap(
    f"data/cache/{config.dataset}/codes/{code_type2}/{code_tokenizer}/{code_length2}/codes.mmp",
    mode="r",
    dtype=np.int32
).reshape(len(text_dataset), -1).copy()

for qidx, positive in positives.items():
    positive = positive[0]
    model1_res = model1_result[qidx]
    model2_res = model2_result[qidx]
    query = query_dataset[qidx]["query"]["input_ids"]
    showcase = False
    if positive in model1_res and positive not in model2_res:
        text_code = text_codes1[positive]
        false_pos_code = text_codes2[model2_res[0]]
        string = f"Model1 wins: {qidx, positive}"
        showcase = True
    
    elif positive in model2_res and positive not in model1_res:
        text_code = text_codes2[positive]
        false_pos_code = text_codes1[model1_res[0]]
        string = f"Model2 wins: {qidx, positive}"
        showcase = True

    if showcase:
        print(f"{string:*^50}")
        print(f"{'Query': <30}: {t.decode(query, skip_special_tokens=True)}")
        print(f"{'Target Anchor': <30}: {t.decode(text_code[text_code != -1], skip_special_tokens=True)}")
        string = f"False Positive Anchor"
        print(f"{string: <30}: {t.decode(false_pos_code[false_pos_code != -1], skip_special_tokens=True)}")

        all_q = [[] for _ in query_sets]
        for query_set_idx, query_idx in queries[positive]:
            query = t.decode(query_datasets[query_set_idx][query_idx]["query"]["input_ids"], skip_special_tokens=True)
            all_q[query_set_idx].append(query)
        for j, query in enumerate(all_q):
            if len(query):
                print(f"{query_sets[j] + ' Queries': <30}: {query}")        
        x = input()
        if x == "s":
            break

In [ ]:
# check the document frequency of each code position
code_type = "bm25_comma-sample"
code_tokenizer = "t5"
code_length = 26
wordset = WordSetIndex(save_dir=f"data/cache/{config.dataset}/codes/{code_type}/{code_tokenizer}/{code_length}", sep_token_id=6)
wordset.fit(None)

max_word_num = (wordset.docs != -1).sum(-1).max()
doc_nums = np.zeros(max_word_num, dtype=np.int32)
valid_nums = np.zeros(max_word_num, dtype=np.int32)

for i in range(max_word_num):
    i_th_position = wordset.docs[:, i]
    valid_nums[i] = (i_th_position != -1).sum()
    i_th_position = i_th_position[i_th_position != -1]
    inverted_lists = wordset.inverted_lists[i_th_position]
    doc_nums[i] = sum([len(x) for x in inverted_lists])
wordset.inverse_vocab.shape, doc_nums / valid_nums

In [ ]:
# create pseudo-queries from document
dataset = "MSMARCO-passage"
query_set = "doc"
text_col = [2]
query_length = 32

with open(f"{config.data_root}/{dataset}/collection.tsv") as collection_file, open(f"{config.data_root}/{dataset}/queries.{query_set}.tsv", "w") as query_file, open(f"{config.data_root}/{dataset}/qrels.{query_set}.tsv", "w") as qrel_file:
    for tidx, line in enumerate(tqdm(collection_file)):
        fields = line.split("\t")
        tid = fields[0]
        query_fields = [field.strip() for col_idx, field in enumerate(fields) if col_idx in text_col and len(field) > 1]
        # maximum number of words
        query = " ".join(query_fields).split(" ")[:query_length]
        query = " ".join(query)

        query_file.write("\t".join([str(tidx), query]) + "\n")
        qrel_file.write("\t".join([str(tidx), "0", tid, "1"]) + "\n")

In [ ]:
# filter query sets with ANCE
ori_query_set = "doct5-5"
query_set = "doct5-5-filter"

filter_model = "ANCE"
filter_results = load_pickle(f"data/cache/{config.dataset}/retrieve/{filter_model}/{ori_query_set}/retrieval_result.pkl")
filter_results = {k: v[:10] for k, v in filter_results.items()}

tid2idx = load_pickle(f"data/cache/{config.dataset}/dataset/text/id2index.pkl")
qid2idx = load_pickle(f"data/cache/{config.dataset}/dataset/query/{ori_query_set}/id2index.pkl")

with open(f"{config.data_root}/{config.dataset}/qrels.{ori_query_set}.tsv") as ori_qrel_file, open(f"{config.data_root}/{config.dataset}/queries.{ori_query_set}.tsv") as ori_query_file, open(f"{config.data_root}/{config.dataset}/qrels.{query_set}.tsv", "w") as qrel_file, open(f"{config.data_root}/{config.dataset}/queries.{query_set}.tsv", "w") as query_file:
    qindices = set()
    for i, line in enumerate(ori_qrel_file):
        qid, _, tid, _ = line.strip().split("\t")
        qidx = qid2idx[qid]
        tidx = tid2idx[tid]
        if tidx in filter_results[qidx]:
            qrel_file.write(line)
            qindices.add(qidx)
    for i, line in enumerate(ori_query_file):
        if i in qindices:
            query_file.write(line)

In [ ]:
# filter MSMARCO Top300k

# filter_indices = []
# for dup_idx in dup_indices:
#     text = t.decode(text_dataset[dup_idx]["text"]["input_ids"], skip_special_tokens=True)
#     text_length = len(text.split(" "))
#     if text_length < 100:
#         filter_indices.append(dup_idx)

filter_indices = set(np.argwhere(df.duplicated().to_numpy())[:, 0].tolist())

with open("/share/project/peitian/Data/Adon/Top300k/collection.tsv") as ori_collection, open("/share/project/peitian/Data/Adon/Top300k/qrels.train.tsv") as ori_train_qrels, open("/share/project/peitian/Data/Adon/Top300k/qrels.dev.tsv") as ori_dev_qrels, open("/share/project/peitian/Data/Adon/Top300k-filter/collection.tsv", "w") as collection, open("/share/project/peitian/Data/Adon/Top300k-filter/qrels.train.tsv", "w") as train_qrels, open("/share/project/peitian/Data/Adon/Top300k-filter/qrels.dev.tsv", "w") as dev_qrels:
    # shutil.copy("/share/project/peitian/Data/Adon/Top300k/queries.train.tsv", "/share/project/peitian/Data/Adon/Top300k-filter/queries.train.tsv")
    # shutil.copy("/share/project/peitian/Data/Adon/Top300k/queries.dev.tsv", "/share/project/peitian/Data/Adon/Top300k-filter/queries.dev.tsv")

    for i, line in enumerate(ori_collection):
        if i not in filter_indices:
            collection.write(line)

    tid2idx = load_pickle("/share/project/peitian/Code/Uni-Retriever/src/data/cache/Top300k/dataset/text/id2index.pkl")
    for line in ori_train_qrels:
        qid, _, tid, _ = line.strip().split()
        tidx = tid2idx[tid]
        if tidx in filter_indices:
            continue
        train_qrels.write(line)
    
    for line in ori_dev_qrels:
        qid, _, tid, _ = line.strip().split()
        tidx = tid2idx[tid]
        if tidx in filter_indices:
            continue
        dev_qrels.write(line)

In [ ]:
# convert yujia DSI code to my format

tid2idx = load_pickle("/share/project/peitian/Code/Uni-Retriever/src/data/cache/Top300k-filter/dataset/text/id2index.pkl")

code_type = "DSI-semantic"
code_tokenizer = "t5"
code_length = 10

t = AutoTokenizer.from_pretrained(os.path.join(config.plm_root, code_tokenizer))
text_codes = np.memmap(
    makedirs(f"data/cache/{config.dataset}/codes/{code_type}/{code_tokenizer}/{code_length}/codes.mmp"),
    mode="w+",
    dtype=np.int32,
    shape=(len(text_dataset), code_length)
)
text_codes[:, 0] = 0
text_codes[:, 1:] = -1

count = 0
with open("/share/project/webbrain-zhouyujia/transfer/data/encoded_docid/t5_semantic_structured_top_300k.txt") as f:
    index = 0
    for line in tqdm(f):
        tid, code = line.strip().split()
        tid = tid[1:-1].upper()
        if tid in tid2idx:
            count += 1
            code = [int(x) for x in code.split(",")]
            text_codes[index, 1:len(code)+1] = code
            index += 1

In [ ]:
# convert yujia ultron code to my format

dataset = "Rand300k-filter"
tid2idx = load_pickle(f"/share/project/peitian/Code/Uni-Retriever/src/data/cache/{dataset}/dataset/text/id2index.pkl")

code_type = "ultron"
code_tokenizer = "t5"
code_length = 34

config = Config()
with initialize(version_base=None, config_path="../data/config/"):
    overrides = [
        f"base={dataset}",
    ]
    hydra_config = compose(config_name="_example", overrides=overrides)
    config._from_hydra(hydra_config)
text_dataset = TextDataset(config)
# t = AutoTokenizer.from_pretrained(os.path.join(config.plm_root, code_tokenizer))
text_codes = np.memmap(
    makedirs(f"data/cache/{dataset}/codes/{code_type}/{code_tokenizer}/{code_length}/codes.mmp"),
    mode="w+",
    dtype=np.int32,
    shape=(len(text_dataset), code_length)
)
text_codes[:, 0] = 0
text_codes[:, 1:] = -1

count = 0
with open("/share/project/webbrain-zhouyujia/transfer/data/encoded_docid/t5_url_title_rand_300k.txt") as f:
    index = 0
    for line in tqdm(f):
        tid, code = line.strip().split()
        tid = tid[1:-1].upper()
        if tid in tid2idx:
            count += 1
            code = [int(x) for x in code.split(",")]
            if len(code) > 33:
                code = code[:33]
                code[-1] = 1
            text_codes[index, 1:len(code)+1] = code
            index += 1

In [ ]:
# convert yujia doct5 to my format

tid2idx = load_pickle("/share/project/peitian/Code/Uni-Retriever/src/data/cache/Top300k-filter/dataset/text/id2index.pkl")

with open("/share/project/webbrain-zhouyujia/transfer/data/msmarco-data/fake_query_10_all.txt") as f, open("/share/project/peitian/Data/Adon/Top300k-filter/queries.doct5.tsv", "w") as query_file, open("/share/project/peitian/Data/Adon/Top300k-filter/qrels.doct5.tsv", "w") as qrel_file:
    for i, line in enumerate(tqdm(f)):
        tid, query = line.split("\t")
        tid = tid[1:-1].upper()

        qid = str(i)
        if tid in tid2idx:
            query_file.write("\t".join([qid, query]))
            qrel_file.write("\t".join([qid, "0", tid, "1"]) + "\n")

In [ ]:
# merge ultron code
code_type = "ultron"
code_tokenizer = "t5"
code_length = 34
text_codes_rand = np.memmap(
    makedirs(f"data/cache/Rand300k-filter/codes/{code_type}/{code_tokenizer}/{code_length}/codes.mmp"),
    mode="r",
    dtype=np.int32
).reshape(-1, code_length).copy()
text_codes_top = np.memmap(
    makedirs(f"data/cache/Top300k-filter/codes/{code_type}/{code_tokenizer}/{code_length}/codes.mmp"),
    mode="r",
    dtype=np.int32
).reshape(-1, code_length).copy()

print(text_codes_rand.shape, text_codes_top.shape)

config = Config()
with initialize(version_base=None, config_path="../data/config/"):
    overrides = [
        f"base=MS600k",
    ]
    hydra_config = compose(config_name="_example", overrides=overrides)
    config._from_hydra(hydra_config)
text_dataset = TextDataset(config)
text_codes = np.memmap(
    makedirs(f"data/cache/MS600k/codes/{code_type}/{code_tokenizer}/{code_length}/codes.mmp"),
    mode="w+",
    dtype=np.int32,
    shape=(len(text_dataset), code_length)
)
text_codes[:, 0] = 0
text_codes[:, 1:] = -1

text_codes[:len(text_codes_rand)] = text_codes_rand
text_codes[len(text_codes_rand):] = text_codes_top

In [ ]:
# split code from MS600k

# code_type = "ANCE_hier"
code_type = "NCI-bias"
code_tokenizer = "t5"
code_length = 10
# code_type_split = "ANCE_hier_600k"
code_type_split = "NCI_600k-bias"

config = Config()
with initialize(version_base=None, config_path="../data/config/"):
    overrides = [
        f"base=MS600k",
    ]
    hydra_config = compose(config_name="_example", overrides=overrides)
    config._from_hydra(hydra_config)
text_dataset = TextDataset(config)

text_codes = np.memmap(
    makedirs(f"data/cache/MS600k/codes/{code_type}/{code_tokenizer}/{code_length}/codes.mmp"),
    mode="r",
    dtype=np.int32,
).reshape(text_dataset.text_num, code_length).copy()

config = Config()
with initialize(version_base=None, config_path="../data/config/"):
    overrides = [
        f"base=Rand300k-filter",
    ]
    hydra_config = compose(config_name="_example", overrides=overrides)
    config._from_hydra(hydra_config)
rand_text_dataset = TextDataset(config)

text_codes_rand = np.memmap(
    makedirs(f"data/cache/Rand300k-filter/codes/{code_type_split}/{code_tokenizer}/{code_length}/codes.mmp"),
    mode="w+",
    dtype=np.int32,
    shape=(rand_text_dataset.text_num, code_length),
)
text_codes_top = np.memmap(
    makedirs(f"data/cache/Top300k-filter/codes/{code_type_split}/{code_tokenizer}/{code_length}/codes.mmp"),
    mode="w+",
    dtype=np.int32,
    shape=(text_dataset.text_num - text_codes_rand.shape[0], code_length)    
)

text_codes_rand[:] = text_codes[:rand_text_dataset.text_num]
text_codes_top[:] = text_codes[rand_text_dataset.text_num:]

In [ ]:
# split some queries

ori_dataset = "NQ"
dataset = "NQ-50k-seen"
query_set = "doct5"

tid2idx = load_pickle(f"data/cache/{dataset}/dataset/text/id2index.pkl")
qids = set()

with \
    open(f"{config.data_root}/{ori_dataset}/qrels.{query_set}.tsv") as ori_qrel_file, \
    open(f"{config.data_root}/{dataset}/qrels.{query_set}.tsv", "w") as qrel_file, \
    open(f"{config.data_root}/{ori_dataset}/queries.{query_set}.tsv") as ori_query_file, \
    open(f"{config.data_root}/{dataset}/queries.{query_set}.tsv", "w") as query_file:
    for i, line in enumerate(ori_qrel_file):
        qid, _, tid, _ = line.strip().split("\t")
        if tid in tid2idx:
            qids.add(qid)
            qrel_file.write(line)
    
    for line in ori_query_file:
        qid, query = line.split("\t")
        if qid in qids:
            query_file.write(line)


In [ ]:
# convert t5 code to bert code

code_type = "chat"
# code_type = "words-weight"
code_tokenizer = "t5"
code_length = 50
# code_length = 34

text_codes = np.memmap(
    f"data/cache/{config.dataset}/codes/{code_type}/{code_tokenizer}/{code_length}/codes.mmp",
    mode="r",
    dtype=np.int32
).reshape(len(text_dataset), -1).copy()

new_code_tokenizer = "bert"
new_t = AutoTokenizer.from_pretrained(os.path.join(config.plm_root, new_code_tokenizer))

new_text_codes = []
max_length = 0
for text_code in tqdm(text_codes):
    text_code = text_code[text_code != -1]
    decoded = t.decode(text_code, skip_special_tokens=True)
    encoded = new_t.encode(decoded, padding=False)
    if new_code_tokenizer == "bert":
        encoded = new_t.encode(decoded, padding=False)[1:]
    new_text_codes.append(encoded)
    if len(encoded) > max_length:
        max_length = len(encoded)

# plus one for the leading padding token
mmp_path = f"data/cache/{config.dataset}/codes/{code_type}/{new_code_tokenizer}/{max_length + 1}/codes.mmp"
makedirs(mmp_path)
new_codes_mmp = np.memmap(
    mmp_path,
    mode="w+",
    dtype=np.int32,
    shape=(len(text_dataset), max_length + 1)
)
new_codes_mmp[:,1:] = -1

for qrel_idx, code in enumerate(tqdm(new_text_codes)):
    new_codes_mmp[qrel_idx, 1:len(code) + 1] = code

print(f"saving at {mmp_path}")

In [ ]:
# reorder words-comma in ascending df

new_docs = np.zeros_like(wordset.docs) - 1
for i, doc in enumerate(tqdm(wordset.docs)):
    doc = doc[doc != -1]
    inverted_lists = wordset.inverted_lists[doc]
    dfs = [len(x) for x in inverted_lists]

    new_doc = sorted(zip(doc, dfs), key=lambda x: x[1])
    new_doc = [x[0] for x in new_doc]
    new_docs[i, :len(new_doc)] = new_doc

code_type = "words-comma-df"
path = f"data/cache/{config.dataset}/codes/{code_type}/{code_tokenizer}/{code_length}/codes.mmp"
makedirs(path)

print(f"saving at {path}...")

new_text_codes = np.memmap(
    path,
    mode="w+",
    dtype=np.int32,
    shape=text_codes.shape
)
new_text_codes[:, 0] = text_codes[:, 0]
new_text_codes[:, 1:] = -1

sep_token_id = int(text_codes[0][text_codes[0] != -1][-1])

for i, new_doc in enumerate(tqdm(new_docs)):
    new_doc = new_doc[new_doc != -1]
    tokens = wordset.inverse_vocab[new_doc].reshape(-1)
    tokens = tokens[tokens != -1].tolist()
    tokens.append(sep_token_id)
    new_text_codes[i, 1: len(tokens) + 1] = tokens

In [ ]:
# add new documents to Top300k-filter
tid2index = load_pickle("/share/project/peitian/Code/Uni-Retriever/src/data/cache/Top300k-filter/dataset/text/id2index.pkl")

with \
    open("/share/project/peitian/Data/Adon/Rand300k-filter/collection.backup.tsv") as f, \
    open("/share/project/peitian/Data/Adon/Rand300k-filter/collection.tsv", "w") as g, \
    open("/share/project/peitian/Data/Adon/Rand300k-filter/qrels.dev.backup.tsv") as dev_qrel, \
    open("/share/project/peitian/Data/Adon/Rand300k-filter/qrels.dev.tsv", "w") as new_dev_qrel:

    tids = []
    for i, line in enumerate(f):
        tid = line.split("\t")[0]
        if tid not in tid2index:
            tids.append(tid)
            g.write(line)
    
    for line in dev_qrel:
        qid, _, tid, _ = line.strip().split()
        if tid in tids:
            new_dev_qrel.write(line)

In [ ]:
tid2index = load_pickle("/share/project/peitian/Code/Uni-Retriever/src/data/cache/Rand300k-filter/dataset/text/id2index.pkl")
tindex2id = {v: k for k, v in tid2index.items()}
dup_tids = {tindex2id[x] for x in dup_indices}

# filtered 300k
with \
    open("/share/project/peitian/Data/Adon/Rand300k-filter/collection.backup.tsv") as collection_file, \
    open("/share/project/peitian/Data/Adon/Rand300k-filter/collection.tsv", "w") as new_collection_file, \
    open("/share/project/peitian/Data/Adon/Rand300k-filter/qrels.dev.backup.tsv") as dev_qrel, \
    open("/share/project/peitian/Data/Adon/Rand300k-filter/qrels.dev.tsv", "w") as new_dev_qrel:

    tids = []
    for i, line in enumerate(collection_file):
        tid = line.split("\t")[0]
        if tid in tid2index and tid not in dup_tids:
            tids.append(tid)
            new_collection_file.write(line)
    
    for line in dev_qrel:
        qid, _, tid, _ = line.strip().split()
        if tid in tids:
            new_dev_qrel.write(line)

In [ ]:
tid2index = load_pickle("data/cache/Rand300k-filter/dataset/text/id2index.pkl")
tindex2id = {v: k for k, v in tid2index.items()}

positives = load_pickle("data/cache/Rand300k-filter/dataset/query/dev/positives.pkl")
positives = set([v[0] for v in positives.values()])
positive_tids = [tindex2id[x] for x in positives]
complement = sample(list([k for k, v in tid2index.items() if v not in positives]), 100000 - len(positives))
tids = set(positive_tids + complement)
print(len(tids))

# filtered 300k
with \
    open("/share/project/peitian/Data/Adon/Rand300k-filter/collection.backup.tsv") as collection_file, \
    open("/share/project/peitian/Data/Adon/Rand100k-filter/collection.tsv", "w") as new_collection_file, \
    open("/share/project/peitian/Data/Adon/Rand300k-filter/qrels.dev.backup.tsv") as dev_qrel, \
    open("/share/project/peitian/Data/Adon/Rand100k-filter/qrels.dev.tsv", "w") as new_dev_qrel, \
    open("/share/project/peitian/Data/Adon/Rand300k-filter/queries.dev.backup.tsv") as dev_query, \
    open("/share/project/peitian/Data/Adon/Rand100k-filter/queries.dev.tsv", "w") as new_dev_query:

    for i, line in enumerate(collection_file):
        tid = line.split("\t")[0]
        if tid in tids:
            tids.add(tid)
            new_collection_file.write(line)

    qids = set()
    for line in dev_qrel:
        qid, _, tid, _ = line.strip().split()
        if tid in tids:
            new_dev_qrel.write(line)
            qids.add(qid)

    for line in dev_query:
        qid = line.strip().split()[0]
        if qid in qids:
            new_dev_query.write(line)

In [ ]:
with \
    open("/share/project/peitian/Data/Adon/Rand300k-filter/collection.tsv") as rand_collection_file, \
    open("/share/project/peitian/Data/Adon/Top300k-filter/collection.tsv") as top_collection_file, \
    open("/share/project/peitian/Data/Adon/MS600k/collection.tsv", "w") as new_collection_file, \
    open("/share/project/peitian/Data/Adon/Rand300k-filter/qrels.dev.tsv") as rand_dev_qrel, \
    open("/share/project/peitian/Data/Adon/Top300k-filter/qrels.dev.tsv") as top_dev_qrel, \
    open("/share/project/peitian/Data/Adon/MS600k/qrels.dev.tsv", "w") as new_dev_qrel, \
    open("/share/project/peitian/Data/Adon/Rand300k-filter/queries.dev.tsv") as rand_dev_query, \
    open("/share/project/peitian/Data/Adon/Top300k-filter/queries.dev.tsv") as top_dev_query, \
    open("/share/project/peitian/Data/Adon/MS600k/queries.dev.tsv", "w") as new_dev_query:

    rand_tids = []
    for line in rand_collection_file:
        tid = line.split("\t")[0]
        rand_tids.append(tid)
        new_collection_file.write(line)
    
    top_tids = []
    for line in top_collection_file:
        tid = line.split("\t")[0]
        top_tids.append(tid)
        new_collection_file.write(line)

    assert len(set(rand_tids).intersection(set(top_tids))) == 0

    for line in rand_dev_query:
        new_dev_query.write(line)
    for line in top_dev_query:
        new_dev_query.write(line)
    for line in rand_dev_qrel:
        new_dev_qrel.write(line)
    for line in top_dev_qrel:
        new_dev_qrel.write(line)


In [ ]:
# split NQ to Seen and Unseen for evaluating performance of adding new documents
# d0_tindices, d0_train_positives, d0_dev_positives, d0_test_positives (3915)
# d1_tindices, d1_test_positives (3915)

train_positives = load_pickle(f"data/cache/{config.dataset}/dataset/query/train/positives.pkl")
test_positives = load_pickle(f"data/cache/{config.dataset}/dataset/query/dev/positives.pkl")

tidx_2_train_query = defaultdict(list)
for k, v in train_positives.items():
    tidx_2_train_query[v[0]].append(k)
tindices_with_train_query = set([x[0] for x in train_positives.values()])

tidx_2_test_query = defaultdict(list)
for k, v in test_positives.items():
    tidx_2_test_query[v[0]].append(k)
tindices_with_test_query = set([x[0] for x in test_positives.values()])

tindices = set(range(len(text_dataset)))

test_qindices = set(test_positives.keys())
d1_test_positives = {}
# unseen documents on the entire training set
d1_tindices = tindices - tindices_with_train_query
for tidx in d1_tindices:
    qindices = tidx_2_test_query[tidx]
    for qidx in qindices:
        d1_test_positives[qidx] = [tidx]

# sampled queries
d1_test_candidates = test_qindices - set(d1_test_positives.keys())
# only keep the queries with 1 relevant document
d1_test_qindices = set()
for qidx in d1_test_candidates:
    if len(tidx_2_test_query[test_positives[qidx][0]]) == 1:
        d1_test_qindices.add(qidx)
d1_test_candidates = random.sample(list(d1_test_qindices), 3915 - len(d1_test_positives))
for qidx in d1_test_candidates:
    d1_test_positives[qidx] = test_positives[qidx]

d1_tindices = set([x[0] for x in d1_test_positives.values()])
# sampled documents relevant to some train queries
d1_candidates = tindices_with_train_query - tindices_with_test_query
d1_tindices.update(random.sample(list(d1_candidates), 50000 - len(d1_tindices)))

d0_test_positives = {}
d0_test_tindices = tindices_with_test_query - d1_tindices
for tidx in d0_test_tindices:
    qindices = tidx_2_test_query[tidx]
    for qidx in qindices:
        d0_test_positives[qidx] = [tidx]

# sanity
d0_tindices = tindices - d1_tindices
for tidx in d0_tindices:
    assert tidx in tidx_2_train_query
assert len(d0_test_positives) == 3915
assert d0_test_tindices - d0_tindices == set()

d0_train_positives = {}
d0_dev_positives = {}

d0_qindices = set()
d0_tindices_with_multiple_rel = set()
for tidx in d0_tindices:
    qindices = tidx_2_train_query[tidx]
    d0_qindices.update(qindices)
    if len(qindices) > 1:
        d0_tindices_with_multiple_rel.add(tidx)

d0_dev_tindices = random.sample(list(d0_tindices_with_multiple_rel), 3000)
for tidx in d0_dev_tindices:
    qindices = tidx_2_train_query[tidx]
    assert len(qindices) > 1
    dev_qidx = random.sample(qindices, 1)[0]
    d0_dev_positives[dev_qidx] = [tidx]
    d0_qindices.remove(dev_qidx)

for qidx in d0_qindices:
    d0_train_positives[qidx] = train_positives[qidx]

In [ ]:
tidx2id = {v: k for k, v in load_pickle(f"data/cache/{config.dataset}/dataset/text/id2index.pkl").items()}
# 50k
d0_dataset = "NQ-50k-seen"
# the remaining 50k
d1_dataset = "NQ-50k-unseen"

train_qidx2id = {v: k for k, v in load_pickle(f"data/cache/{config.dataset}/dataset/query/train/id2index.pkl").items()}
dev_qidx2id = {v: k for k, v in load_pickle(f"data/cache/{config.dataset}/dataset/query/dev/id2index.pkl").items()}

os.makedirs(f"{config.data_root}/{d0_dataset}", exist_ok=True)
os.makedirs(f"{config.data_root}/{d1_dataset}", exist_ok=True)

# with open(f"{config.data_root}/{config.dataset}/collection.tsv") as collection_file, \
#     open(f"{config.data_root}/{config.dataset}/queries.train.tsv") as train_query_file, \
#     open(f"{config.data_root}/{config.dataset}/queries.dev.tsv") as dev_query_file, \
#     open(f"{config.data_root}/{d0_dataset}/collection.tsv", "w") as d0_collection_file, \
#     open(f"{config.data_root}/{d0_dataset}/queries.train.tsv", "w") as d0_train_query_file, \
#     open(f"{config.data_root}/{d0_dataset}/queries.dev.tsv", "w") as d0_dev_query_file, \
#     open(f"{config.data_root}/{d0_dataset}/queries.test.tsv", "w") as d0_test_query_file, \
#     open(f"{config.data_root}/{d0_dataset}/qrels.train.tsv", "w") as d0_train_qrel_file, \
#     open(f"{config.data_root}/{d0_dataset}/qrels.dev.tsv", "w") as d0_dev_qrel_file, \
#     open(f"{config.data_root}/{d0_dataset}/qrels.test.tsv", "w") as d0_test_qrel_file, \
#     open(f"{config.data_root}/{d1_dataset}/collection.tsv", "w") as d1_collection_file, \
#     open(f"{config.data_root}/{d1_dataset}/queries.dev.tsv", "w") as d1_test_query_file, \
#     open(f"{config.data_root}/{d1_dataset}/qrels.dev.tsv", "w") as d1_test_qrel_file:

#     for tidx, line in enumerate(tqdm(collection_file)):
#         if tidx in d0_tindices:
#             d0_collection_file.write(line)
#         if tidx in d1_tindices:
#             d1_collection_file.write(line)
    
#     for qidx, line in enumerate(tqdm(train_query_file)):
#         if qidx in d0_train_positives:
#             positive = d0_train_positives[qidx][0]
#             d0_train_query_file.write(line)
#             d0_train_qrel_file.write("\t".join([train_qidx2id[qidx], "0", tidx2id[positive], "1"]) + "\n")
#         elif qidx in d0_dev_positives:
#             positive = d0_dev_positives[qidx][0]
#             d0_dev_query_file.write(line)
#             d0_dev_qrel_file.write("\t".join([train_qidx2id[qidx], "0", tidx2id[positive], "1"]) + "\n")

#     for qidx, line in enumerate(tqdm(dev_query_file)):
#         if qidx in d0_test_positives:
#             positive = d0_test_positives[qidx][0]
#             d0_test_query_file.write(line)
#             d0_test_qrel_file.write("\t".join([dev_qidx2id[qidx], "0", tidx2id[positive], "1"]) + "\n")
#         if qidx in d1_test_positives:
#             positive = d1_test_positives[qidx][0]
#             d1_test_query_file.write(line)
#             d1_test_qrel_file.write("\t".join([dev_qidx2id[qidx], "0", tidx2id[positive], "1"]) + "\n")

In [ ]:
# code_types = ["ANCE_hier", "words_comma", "words_comma_plus_stem", "title"]
code_types = ["id"]
code_lengths = [8]
code_tokenizer = "t5"

for code_type, code_length in zip(code_types, code_lengths):
    text_codes = np.memmap(
        f"data/cache/{config.dataset}/codes/{code_type}/{code_tokenizer}/{code_length}/codes.mmp",
        mode="r",
        dtype=np.int32
    ).reshape(len(text_dataset), code_length).copy()

    d0_text_codes = np.memmap(
        makedirs(f"data/cache/{d0_dataset}/codes/{code_type}/{code_tokenizer}/{code_length}/codes.mmp"),
        mode="w+",
        dtype=np.int32,
        shape=(len(d0_tindices), code_length)
    )
    d1_text_codes = np.memmap(
        makedirs(f"data/cache/{d1_dataset}/codes/{code_type}/{code_tokenizer}/{code_length}/codes.mmp"),
        mode="w+",
        dtype=np.int32,
        shape=(len(d1_tindices), code_length)
    )
    counts = [0 for _ in range(4)]
    for tidx, text_code in enumerate(text_codes):
        if tidx in d0_tindices:
            d0_text_codes[counts[0]] = text_code
            counts[0] += 1
        if tidx in d1_tindices:
            d1_text_codes[counts[1]] = text_code
            counts[1] += 1
    assert all([counts[i] == len(eval(f"d{i}_tindices")) for i in range(2)])

In [ ]:
# case for generation likelihood
model = AM.from_pretrained("/share/project/peitian/Code/Uni-Retriever/src/data/cache/NQ/ckpts/BOW_qg/first+random3+em", device="cpu")

In [ ]:
query = default_collate([query_dataset[3315]["query"]])

text_code = [0]
# for x in "executive office of president".split(" "):
# for x in "white house communication director".split(" "):
for x in "cristeta comerford executive chef".split(" "):
# for x in "executive chef white house".split(" "):
# for x in "white house executive chef".split(" "):
    text_code += t.encode(x, add_special_tokens=False) + [6]
text_code += [1]
text_code = default_collate([text_code])

# text_code = default_collate([text_codes[108361].astype(np.int64)])

text_code[text_code == -1] = 0
display(t.decode(text_code[0]))

with torch.no_grad():
    logits = model.plm(**query, decoder_input_ids=text_code).logits
    logits = torch.log_softmax(logits, dim=-1)
    logits = logits.gather(dim=-1, index=text_code[:, 1:, None])[0,:,0]
    # logits[4] = logits[4] - 4
    cum = logits.cumsum(dim=-1).numpy().round(3)
    tokens = t.convert_ids_to_tokens(text_code[0])[1:]
list(zip(cum, tokens))

In [ ]:
# interleave two query sets to form a unified one with all documents having the same number of queries

k = 5
dataset = "NQ"
query_set = "pad5"
main_query_set = "train"
sup_query_set = "doct5"

main_qid2idx = load_pickle(f"data/cache/{dataset}/dataset/query/{main_query_set}/id2index.pkl")
sup_qid2idx = load_pickle(f"data/cache/{dataset}/dataset/query/{sup_query_set}/id2index.pkl")
tid2idx = load_pickle(f"data/cache/{dataset}/dataset/text/id2index.pkl")

with \
    open(f"{config.data_root}/{dataset}/qrels.{main_query_set}.tsv") as main_qrel_file, \
    open(f"{config.data_root}/{dataset}/qrels.{sup_query_set}.tsv") as sup_qrel_file, \
    open(f"{config.data_root}/{dataset}/qrels.{query_set}.tsv", "w") as qrel_file, \
    open(f"{config.data_root}/{dataset}/queries.{main_query_set}.tsv") as main_query_file, \
    open(f"{config.data_root}/{dataset}/queries.{sup_query_set}.tsv") as sup_query_file, \
    open(f"{config.data_root}/{dataset}/queries.{query_set}.tsv", "w") as query_file:

    new_qid = 0
    # map qidx to a new query id
    qidx2newid = {}
    qid2idx = load_pickle(f"data/cache/{dataset}/dataset/query/{main_query_set}/id2index.pkl")
    main_tid2qidx = defaultdict(list)
    for i, line in enumerate(main_qrel_file):
        qid, _, tid, _ = line.strip().split("\t")
        # keep the first k elements
        if len(main_tid2qidx[tid]) < k:
            qidx = qid2idx[qid]
            main_tid2qidx[tid].append(qidx)
            qidx2newid[qidx] = new_qid
            new_qid += 1

    # slice main queries
    for qidx, line in enumerate(main_query_file):
        if qidx in qidx2newid:
            qid, query = line.split("\t")
            qid = qidx2newid[qidx]
            query_file.write("\t".join([str(qid), query]))

    # store qrel using new_qid
    for tid, qindices in main_tid2qidx.items():
        for qidx in qindices:
            qid = qidx2newid[qidx]
            qrel_file.write("\t".join([str(qid), "0", tid, "1"]) + '\n')

    qidx2newid = {}
    sup_tid2qidx = defaultdict(list)
    qid2idx = load_pickle(f"data/cache/{dataset}/dataset/query/{sup_query_set}/id2index.pkl")
    for i, line in enumerate(sup_qrel_file):
        qid, _, tid, _ = line.strip().split("\t")
        # keep the first k elements
        # pad to k
        if len(main_tid2qidx[tid]) + len(sup_tid2qidx[tid]) < k:
            qidx = qid2idx[qid]
            sup_tid2qidx[tid].append(qidx)
            qidx2newid[qidx] = new_qid
            new_qid += 1
    
    # slice sup queries
    for qidx, line in enumerate(sup_query_file):
        if qidx in qidx2newid:
            qid, query = line.split("\t")
            qid = qidx2newid[qidx]
            query_file.write("\t".join([str(qid), query]))

    # store qrel using new_qid
    for tid, qindices in sup_tid2qidx.items():
        for qidx in qindices:
            qid = qidx2newid[qidx]
            qrel_file.write("\t".join([str(qid), "0", tid, "1"]) + '\n')